In [4]:
import numpy as np
import scipy.signal as signal
import pyaudio
import wave
import tkinter as tk
from tkinter import filedialog
import threading
import time

# Thiết lập thông số âm thanh
CHUNK = 2048  # Kích thước mỗi khối âm thanh
FORMAT = pyaudio.paInt16  # Định dạng âm thanh 16-bit
CHANNELS = 1  # Số lượng kênh (mono)
RATE = 44100  # Tần số lấy mẫu (sẽ cập nhật từ file)

# Hàm lọc âm thanh
def apply_equalizer(audio_data, bass_gain, mid_gain, treble_gain, framerate):
    # Lọc Bass (dưới 300Hz)
    bass_filter = signal.firwin(101, cutoff=300, fs=framerate, pass_zero=True)
    bass = signal.lfilter(bass_filter, 1.0, audio_data) * bass_gain

    # Lọc Mid (300Hz - 4000Hz)
    mid_filter = signal.firwin(101, [300, 4000], fs=framerate, pass_zero=False)
    mid = signal.lfilter(mid_filter, 1.0, audio_data) * mid_gain

    # Lọc Treble (trên 4000Hz)
    treble_filter = signal.firwin(101, cutoff=4000, fs=framerate, pass_zero=False)
    treble = signal.lfilter(treble_filter, 1.0, audio_data) * treble_gain

    # Tổng hợp tín hiệu đã cân bằng
    output_data = bass + mid + treble
    output_data = np.clip(output_data, -32768, 32767)  # Giới hạn tín hiệu
    return output_data.astype(np.int16)

# Hàm phát âm thanh trong luồng riêng
def play_audio_with_equalizer(filename, bass_slider, mid_slider, treble_slider, stop_event):
    # Mở file wav
    wf = wave.open(filename, 'rb')
    p = pyaudio.PyAudio()

    # Thông số âm thanh từ file
    framerate = wf.getframerate()
    channels = wf.getnchannels()
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=channels,
                    rate=framerate,
                    output=True)

    # Phát và xử lý theo khối CHUNK
    data = wf.readframes(CHUNK)
    while data and not stop_event.is_set():
        audio_data = np.frombuffer(data, dtype=np.int16)
        
        # Đọc giá trị thanh trượt trong thời gian thực
        bass_gain = bass_slider.get()
        mid_gain = mid_slider.get()
        treble_gain = treble_slider.get()

        # Áp dụng bộ lọc cân bằng
        processed_data = apply_equalizer(audio_data, bass_gain, mid_gain, treble_gain, framerate)
        stream.write(processed_data.tobytes())
        data = wf.readframes(CHUNK)

    # Đóng stream và file
    stream.stop_stream()
    stream.close()
    wf.close()
    p.terminate()

# Giao diện người dùng
def run_equalizer_gui():
    def load_file():
        filename = filedialog.askopenfilename(filetypes=[("Audio Files", "*.wav")])
        if filename:
            file_label.config(text=f"Selected File: {filename}")
            play_button.config(state="normal")
            global selected_file
            selected_file = filename

    def start_playback():
        global stop_event
        stop_event.clear()  # Đảm bảo sự kiện dừng được reset
        if selected_file:
            # Khởi tạo luồng phát âm thanh
            threading.Thread(target=play_audio_with_equalizer, args=(
                selected_file, bass_slider, mid_slider, treble_slider, stop_event)).start()

    def stop_playback():
        stop_event.set()  # Kích hoạt dừng luồng phát

    # Thiết lập giao diện
    root = tk.Tk()
    root.title("Real-Time Audio Equalizer")

    # Nút chọn file
    load_button = tk.Button(root, text="Select Audio File", command=load_file)
    load_button.pack()

    # Nhãn hiển thị file đã chọn
    file_label = tk.Label(root, text="No file selected")
    file_label.pack()

    # Thanh trượt điều chỉnh Bass, Mid, Treble
    bass_slider = tk.Scale(root, from_=0, to=3, resolution=0.1, orient=tk.HORIZONTAL, label="Bass Gain")
    bass_slider.set(1.0)
    bass_slider.pack()

    mid_slider = tk.Scale(root, from_=0, to=3, resolution=0.1, orient=tk.HORIZONTAL, label="Mid Gain")
    mid_slider.set(1.0)
    mid_slider.pack()

    treble_slider = tk.Scale(root, from_=0, to=3, resolution=0.1, orient=tk.HORIZONTAL, label="Treble Gain")
    treble_slider.set(1.0)
    treble_slider.pack()

    # Nút phát và dừng âm thanh
    play_button = tk.Button(root, text="Play Audio", command=start_playback, state="disabled")
    play_button.pack()

    stop_button = tk.Button(root, text="Stop Audio", command=stop_playback)
    stop_button.pack()

    root.mainloop()

# Biến lưu tên file và sự kiện dừng luồng phát
selected_file = None
stop_event = threading.Event()

# Chạy giao diện
run_equalizer_gui()
